In [110]:
#Make sure you have configuration file in the same directory where you are running this ipynb file

In [6]:
import json

In [51]:
with open('config.json','r') as f:
    config_data = f.read()

In [52]:
config = json.loads(config_data)

In [53]:
config['Link']

'https://seekingalpha.com/article/4214731-amazon-com-amzn-q3-2018-results-earnings-call-transcript'

In [54]:
config['MS Azure Region']

'eastus'

In [55]:
link = config['Link']

In [14]:
import urllib3

In [15]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"
    }

In [16]:
http = urllib3.PoolManager()
response = http.request('GET', link, headers = headers)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [17]:
response.status

200

In [19]:
if response.status == 200:
    print('Valid URL')
    #Complete code will start here
else :
    print('Invalid URL')
    #Exit code

Valid URL


In [32]:
#validate AWS Credentials
import botocore.session

In [35]:
access_key = config['AWS Access Keys']
secret_access_key = config['AWS Secret Keys']

In [33]:
session = botocore.session.get_session()

In [36]:
sts = session.create_client('sts', aws_access_key_id=access_key,
                                aws_secret_access_key=secret_access_key)

In [37]:
response = sts.get_caller_identity()

In [40]:
response

{'Account': '515649345368',
 'Arn': 'arn:aws:iam::515649345368:user/nagare.p',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '405',
   'content-type': 'text/xml',
   'date': 'Sat, 17 Nov 2018 08:12:13 GMT',
   'x-amzn-requestid': '7d32fcdb-ea40-11e8-9770-8db9716fceb0'},
  'HTTPStatusCode': 200,
  'RequestId': '7d32fcdb-ea40-11e8-9770-8db9716fceb0',
  'RetryAttempts': 0},
 'UserId': 'AIDAJ5UR4XOY6YNIDVGVQ'}

In [41]:
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('AWS Credentials are valid!')
    #Proceed to next validation
else:
    print('AWS Credentials are Invalid')
    #Exit code

AWS Credentials are valid!


In [56]:
#Validating Microsoft Azure API connections
uri = config['MS Azure Region']+'.api.cognitive.microsoft.com'
accessKey = config['MS Azure Access Keys']

In [58]:
from http.client import HTTPSConnection

In [96]:
test_doc = {'documents' : [
  {'id': '1', 'language': 'en', 'text': 'This is test data!'}
]}

def GetSentiment (documents):
    path = '/text/analytics/v2.0/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = HTTPSConnection (uri)
    body = json.dumps (documents)
    conn.request ("POST", path, body, headers)
    response = conn.getresponse ()
    return response

result = GetSentiment(test_doc)

In [64]:
if result.status == 200:
    print('Microsoft Keys are valid!')
    #Proceed with other validation
else :
    print('Microsoft Keys are Invalid!')
    #Exit code

Microsoft Keys are valid!


In [21]:
from bs4 import BeautifulSoup

In [22]:
doc = BeautifulSoup(response.data)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [23]:
temp = doc.find(id="a-body")

In [24]:
temp.text

"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earnings Call October 25, 2018  5:30 PM ET\nExecutives\nDave Fildes - Amazon.com, Inc.\nBrian T. Olsavsky - Amazon.com, Inc.\nAnalysts\nJustin Post - Bank of America Merrill Lynch\nMark Mahaney - RBC Capital Markets LLC\nDouglas T. Anmuth - JPMorgan Securities LLC\nMark A. May - Citigroup Global Markets, Inc.\nBrian Nowak - Morgan Stanley & Co. LLC\nEric J. Sheridan - UBS Securities LLC\nRoss Sandler - Barclays Capital, Inc.\nYoussef Squali - SunTrust Robinson Humphrey, Inc.\nJason Helfstein - Oppenheimer & Co., Inc.\nAnthony DiClemente - Evercore Group LLC\nLloyd Walmsley - Deutsche Bank Securities, Inc.\nColin Alan Sebastian - Robert W. Baird & Co., Inc.\nOperator\nGood day, everyone, and welcome to the Amazon.com Q3 2018 Financial Results Teleconference. At this time, all participants are in a listen-only mode. After the presentation, we will conduct a question-and-answer session. Today's call is being recorded.\nFor opening remarks, I will be

In [25]:
data = temp.getText().split('\n')

In [26]:
len(data)

145

In [28]:
#This is to remove blanks from the list:
i = 0
while i < len(data):
    if data[i] == '':
        del data[i]
    else:
        i+=1

In [29]:
len(data)

138

In [27]:
#This is to fine the text for each sentance, Also seperate out the questions.
data2 = temp.getText().replace('.\n','. ').replace('\n','. ').replace('?','?. ').split('. ')

In [30]:
#This is to remove blanks from the list:
i = 0
while i < len(data2):
    if data2[i] == '':
        del data2[i]
    else:
        i+=1

In [31]:
len(data2)

420

In [66]:
n = len(data)
dict_q3 = dict(zip(range(1,n+1),data))
dict_sentiment = dict(zip(range(1,n+1),['neutral']*n))
dict_fin = {'text':dict_q3,'sentiment':dict_sentiment}

In [70]:
import boto3
import pandas as pd

In [69]:
comprehend = boto3.client(
    'comprehend',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_access_key,
    region_name='us-east-1'
)

In [108]:
rows = []
i=1
for text in data:
  sentimentData = comprehend.detect_sentiment(Text=text, LanguageCode='en')
  row = [i, 'AWS', sentimentData['Sentiment'],sentimentData['SentimentScore']['Negative'],sentimentData['SentimentScore']['Neutral'],sentimentData['SentimentScore']['Positive']]
  rows.append(row)
  i+=1
result_aws = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Overall', 'Negative', 'Neutral', 'Positive'])

In [109]:
result_aws

,Text_ID,API,Overall,Negative,Neutral,Positive
0,1,AWS,NEUTRAL,0.000141,0.998418,0.001304
1,2,AWS,NEUTRAL,0.051804,0.886653,0.054032
2,3,AWS,NEUTRAL,0.002629,0.968652,0.027480
3,4,AWS,NEUTRAL,0.019780,0.943491,0.033096
4,5,AWS,NEUTRAL,0.027782,0.952745,0.015769
5,6,AWS,NEUTRAL,0.006145,0.990782,0.001982
6,7,AWS,NEUTRAL,0.005342,0.985570,0.008148
7,8,AWS,NEUTRAL,0.002651,0.992066,0.004555
8,9,AWS,NEUTRAL,0.005039,0.985260,0.008707
9,10,AWS,NEUTRAL,0.003828,0.974375,0.020420


In [71]:
#At this point we should analyse the sentiment score from AWS API and finilized it with the threshold scoring!

In [72]:
#Start of Microsoft Azure API Sentinment Analysis

In [73]:
n = len(data)
id=[i for i in range(1,1+n)]
language=['en' for i in range(n)]
document=[]

In [74]:
#Generating form for hitting it on Azure Text Analystics API 
for i in range(n):
    dic = {'id':id[i],'language':language[i],'text':data[i]}
    document.append(dic)
documents = {'documents':document}

In [103]:
def GetSentiment (documents1):
    path = '/text/analytics/v2.0/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = HTTPSConnection (uri)
    body = json.dumps (documents1)
    conn.request ("POST", path, body, headers)
    response = conn.getresponse ()
    return response.read ()
result = GetSentiment(documents)

In [104]:
sentiments = json.loads(result)["documents"]

In [105]:
sentiments

[{'id': '1', 'score': 0.5},
 {'id': '2', 'score': 0.973930835723877},
 {'id': '3', 'score': 0.5},
 {'id': '4', 'score': 0.5},
 {'id': '5', 'score': 0.7526739239692688},
 {'id': '6', 'score': 0.5},
 {'id': '7', 'score': 0.5},
 {'id': '8', 'score': 0.5},
 {'id': '9', 'score': 0.5},
 {'id': '10', 'score': 0.5},
 {'id': '11', 'score': 0.5},
 {'id': '12', 'score': 0.5},
 {'id': '13', 'score': 0.5},
 {'id': '14', 'score': 0.5},
 {'id': '15', 'score': 0.5},
 {'id': '16', 'score': 0.5},
 {'id': '17', 'score': 0.5},
 {'id': '18', 'score': 0.5},
 {'id': '19', 'score': 0.8917422294616699},
 {'id': '20', 'score': 0.5},
 {'id': '21', 'score': 0.5},
 {'id': '22', 'score': 0.5},
 {'id': '23', 'score': 0.5},
 {'id': '24', 'score': 0.5},
 {'id': '25', 'score': 0.5},
 {'id': '26', 'score': 0.8590108752250671},
 {'id': '27', 'score': 0.5},
 {'id': '28', 'score': 0.5},
 {'id': '29', 'score': 0.5},
 {'id': '30', 'score': 0.8830258250236511},
 {'id': '31', 'score': 0.5},
 {'id': '32', 'score': 0.94276559352

In [106]:
i = 0
rows = []
for item in sentiments:
  row = [sentiments[i]['id'], 'Azure', sentiments[i]['score']]
  rows.append(row)
  i+=1
result_azure = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Overall_Score'])

In [107]:
result_azure

,Text_ID,API,Overall_Score
0,1,Azure,0.500000
1,2,Azure,0.973931
2,3,Azure,0.500000
3,4,Azure,0.500000
4,5,Azure,0.752674
5,6,Azure,0.500000
6,7,Azure,0.500000
7,8,Azure,0.500000
8,9,Azure,0.500000
9,10,Azure,0.500000


In [111]:
#To-do add timeit for each interation of the API to evaluate the performance

In [114]:
#Finiliazed about do we have to find sentiment for the sentance or for the paragraph? 
#Also, do we have to seperate the questions from the text?
#Do we have to remove the names, single valued words? This will make API call less expensive!